In [1]:
import pandas as pd
import glob
from os.path import basename
from datetime import date

In [2]:
CODIGO_MATERIA = 'A'
ASIGNATURA = 'PYTHON'
NUMERO_GRUPOS = 73
RUTA = './INPUT/GRUPOS_PREDEFINITIVAS/'
CONSULTA = './INPUT/DEFINITIVAS_4-06.csv'
MATERIA_CON_HOMOLOGACION = False
GRUPO_HOMOLOGACION = 'HOM'
NOMENCLATURA_OMITIR = 5
NOMENCLATURA_TIENE_PUNTOS = False
TOLERANCIA_MOODLE = 0.051
ACTIVIDADES_CALIFICABLES = [
    'Reto tema 1',
    'Reto tema 2',
    'Reto tema 3',
    'Reto tema 4',
    'Reto tema 5',
]

In [3]:
DEFINITIVAS = pd.read_csv(CONSULTA, skiprows=1, sep=';', encoding='latin-1')
DEFINITIVAS = DEFINITIVAS[DEFINITIVAS['ASIGNATURA'] == ASIGNATURA]

In [4]:
DEFINITIVAS

,ANO,PE,MINTIC,ESTUDIANTE,DOCUMENTO,APELLIDOS,NOMBRES,EMAIL,ASIGNATURA,GRUPO,CREDI,NOTA,Unnamed: 12
2,2022,1,20300,2223526,1005175753,ANGEL DANILO,ALVAREZ BARAJAS,danilobarajas72@gmail.com,PYTHON,A53,8,5.0,NaN
5,2022,1,31454,2221267,1098720267,NAYIB ANTONIO,ARGUMEDO TORRES,nayibargumedo@hotmail.com,PYTHON,A60,8,1.3,NaN
8,2022,1,56452,2223444,1000694296,JONATHAN DAVID,AVILA OLAYA,davidavila21022003@gmail.com,PYTHON,A15,8,5.0,NaN
11,2022,1,80668,2224083,1070616175,SERGIO ANDRES,AVILA RODRIGUEZ,ser_1095@outlook.com,PYTHON,A67,8,0.0,NaN
14,2022,1,63014,2220648,1012429603,MARIANA,BOLIVAR SEPULVEDA,marianabolivars@gmail.com,PYTHON,A70,8,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14672,2022,1,74029,2220152,1001283341,HECTOR,ZULUAGA LOPEZ,heczl777@gmail.com,PYTHON,A46,8,5.0,NaN
14675,2022,1,84714,2222789,1000471453,LAURA,ZULUAGA VILLAMIL,laurazuluagavillamil7@gmail.com,PYTHON,A54,8,5.0,NaN
14678,2022,1,47289,2221948,1114877205,ERIK SAMIR,ZUNIGA BETANCOURTH,erimurr27@gmail.com,PYTHON,A23,8,0.0,NaN
14681,2022,1,51604,2223780,1022421536,GENESIS MELIZA,ZUNIGA GONZALEZ,GENESISMZUNIGA8@GMAIL.COM,PYTHON,A56,8,3.8,NaN


In [5]:
def extraerNombre(ruta, punto = False):
    nBase = basename(archivo) 
    tokens = nBase.split('-')
    casiNombre = '-'.join(tokens[NOMENCLATURA_OMITIR:-1])
    nombre = casiNombre.split('.')[0]
    return nombre if not(punto) else casiNombre

def getNotaLibro(notaLibro, rayita = True):
    if notaLibro == '-':
        return notaLibro if rayita else 0.0
    return float(notaLibro)

def buscarEnDf(df, columnaKey, key, columna):
    query = df[df[columnaKey] == key][columna]
    if len(query) > 0:        
        return query.iloc[0]
    return -1

In [6]:
if MATERIA_CON_HOMOLOGACION:
    DEFINITIVAS_HOMOLOGADOS = DEFINITIVAS[DEFINITIVAS['GRUPO'] == GRUPO_HOMOLOGACION]
noEncontrados = []
for i in range (1, NUMERO_GRUPOS + 1):
    num = str(i)
    grupo = CODIGO_MATERIA + num
    rutaCarpeta = RUTA + grupo
    archivosCarpeta = glob.glob(rutaCarpeta + '/*.csv')
    df = -1
    for archivo in archivosCarpeta:
        nombreArchivo = extraerNombre(archivo, NOMENCLATURA_TIENE_PUNTOS)
        if not(nombreArchivo in ACTIVIDADES_CALIFICABLES):
            df = pd.read_csv(archivo)
            break
    if type(df) == int:
        raise Exception('Archivo de calificaciones no encontrado')
    definitivasGrupo = DEFINITIVAS[DEFINITIVAS['GRUPO'] == grupo]
    for index, row in df.iterrows():
        nota = round(getNotaLibro(row['Total del curso (Real)'], False), 1)
        codBase = row['Número de ID'] if type(row['Número de ID']) == str else '-1'
        codigoEst = int(codBase.split('-')[-1])
        notaSA = buscarEnDf(definitivasGrupo, 'ESTUDIANTE', codigoEst, 'NOTA')
        if nota != notaSA:
            homologado = False
            if MATERIA_CON_HOMOLOGACION:
                notaHom = buscarEnDf(DEFINITIVAS_HOMOLOGADOS, 'ESTUDIANTE', codigoEst, 'NOTA')
                if notaHom != -1:
                    homologado = True
            noEncontrados.append([codigoEst, grupo, homologado, nota, notaSA])
        

In [7]:
len(noEncontrados)

22

In [8]:
pd.DataFrame(noEncontrados, columns = ['CÓDIGO', 'GRUPO', 'HOMOLOGADO', 'NOTA_LIBRO', 'NOTA_SISTEMA_ACADÉMICO']).to_excel('./OUTPUT/AUDITORIA_NOTAS_SA_'+ ASIGNATURA + '_' + str(date.today()) + '.xlsx', index = False)